In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random as r
import scipy.stats as S
data = pd.read_csv("datasets/train.csv")


# scaling of data

In [2]:
from sklearn.preprocessing import StandardScaler
standardscalar_x=StandardScaler()
data.iloc[:,1:]=standardscalar_x.fit_transform(data.iloc[:,1:])

In [3]:
digitwisedata=list(map(lambda digit:  data[data['label']==digit].iloc[:,:]  , [0,1,2,3,4,5,6,7,8,9]  ))


In [4]:
digitwise_training_data=list(map(lambda digit: digitwisedata[digit].iloc[:int(.75*len(digitwisedata[digit])),:] , [0,1,2,3,4,5,6,7,8,9] ))

digitwise_testing_data=list(map(lambda digit: digitwisedata[digit].iloc[int(.75*len(digitwisedata[digit])):,:] , [0,1,2,3,4,5,6,7,8,9] ))

In [5]:
training_data=pd.concat(digitwise_training_data)
testing_data=pd.concat(digitwise_testing_data)

In [6]:

x_Train_Data = training_data.iloc[:,1:]
y_Train_Data = training_data.iloc[:,0]

x_Test_Data = testing_data.iloc[:,1:]
y_Test_Data = testing_data.iloc[:,0].values

digitwisedata=list(map(lambda digit:  training_data[training_data['label']==digit].iloc[:,1:]  , [0,1,2,3,4,5,6,7,8,9]  ))
pcap_digitwise = list(map(lambda digit: digitwisedata[digit].shape[0] /training_data.shape[0] ,[0,1,2,3,4,5,6,7,8,9] ))


In [7]:
def PCA(data_to_train,data_to_test):
    cov=np.cov(data_to_train.T)
    eigenValues,eigenVectors=np.linalg.eig(cov)
    eigen_pairs={}
    for i in range(len(eigenValues)):
        eigen_pairs[eigenValues[i]]=eigenVectors[i]
    total_variance=sum(eigenValues)
    percentageVariance=[(variance/total_variance) for variance in sorted(eigenValues,reverse=True) ]
    accumulativeVariance=[]
    accumulativeVariance=np.cumsum(percentageVariance)
    for i in range(len(accumulativeVariance)):
        if accumulativeVariance[i]>=.95:
            breaking_index=i
            break
    projection=[]
    for i in sorted(eigenValues,reverse=True):
        projection.append(np.real(eigen_pairs[i][:breaking_index]))
    projection=np.array(projection)
    X_Train_Data_PCA = data_to_train.dot(projection)

    X_Test_Data_PCA = data_to_test.dot(projection)
    
    return X_Train_Data_PCA,X_Test_Data_PCA

In [8]:
posterior_prob=[]
for digit in range(len(digitwisedata)):
    x_Train_Data_PCA,x_Test_Data_PCA=PCA(digitwisedata[digit],x_Test_Data)
    Mean=np.mean(x_Train_Data_PCA,axis=0)
    Cov=np.cov(x_Train_Data_PCA.T)
    prob=pcap_digitwise[digit]*S.multivariate_normal.pdf(x_Test_Data_PCA,Mean,Cov)
    posterior_prob.append(prob)

In [9]:
posterior_prob=np.array(posterior_prob)
posterior_prob.shape

(10, 10503)

In [10]:
y_predicted=[]
correct_prediction=0
for example in range((posterior_prob.shape[1])):
    y_predicted.append(np.argmax(posterior_prob[:,example]))
    
    if np.argmax(posterior_prob[:,example])==y_Test_Data[example]:
        correct_prediction+=1

In [11]:
accuracy=correct_prediction*100/len(y_Test_Data)

In [12]:
print("Accuracy: ",round(accuracy),"%")

Accuracy:  44 %
